# In this notebook we will take a look at the _Housing Price_ Dataset and implement some Machine Learning Algorithms

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Importing the data